In [1]:
#@title Import relevant modules
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D


In [2]:
#@title Loading the dataset
import pickle
import numpy as np
pickle_in = open("X.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y_train = pickle.load(pickle_in)

y_train=np.array(y_train)
X_train = X_train/255.0  #Normalization

#print(X_train[200][10]) # Output a normalized row

In [3]:
#@title Define the plotting function
def plot_curve(epochs, hist, list_of_metrics):
  """Plot a curve of one or more classification metrics vs. epoch."""  
  # list_of_metrics should be one of the names shown in:
  # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics  

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Value")

  for m in list_of_metrics:
    x = hist[m]
    plt.plot(epochs[1:], x[1:], label=m)

  plt.legend()

print("Loaded the plot_curve function.")

Loaded the plot_curve function.


In [4]:
def create_model(my_learning_rate):
  """Create and compile a deep neural net."""
  
  # All models in this course are sequential.
  model = tf.keras.models.Sequential()

  # The features are stored in a two-dimensional 200X200 array. 
  # Flatten that two-dimensional array into a a one-dimensional 
  model.add(tf.keras.layers.Flatten(input_shape=(200, 200, 1)))
  

  # Define the first hidden layer.   
  model.add(tf.keras.layers.Dense(units=64, activation='relu'))

  # Define the second hidden layer. 
  model.add(tf.keras.layers.Dense(units=64, activation='relu'))
  
  # Define a dropout regularization layer. 
  model.add(tf.keras.layers.Dropout(rate=0.08))


  model.add(tf.keras.layers.Dense(units=3, activation='softmax'))     
                           
  # Construct the layers into a model that tensorflow can execute.  
  # Notice that the loss function for multi-class classification
  # is different than the loss function for binary classification.  
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=my_learning_rate),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
  
  return model
  


def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.2):
  """Train the model by feeding it data."""

  history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                      epochs=epochs, shuffle=True, 
                      validation_split=validation_split)
 
  # To track the progression of training, gather a snapshot
  # of the model's metrics at each epoch. 
  epochs = history.epoch
  hist = pd.DataFrame(history.history)

  model.save('CNN.model')
  model.summary

  return epochs, hist    

In [5]:
# The following variables are the hyperparameters.
#learning_rate = 0.003
learning_rate = 0.00001
epochs = 120
batch_size = 32
validation_split = 0.2

# Establish the model's topography.
my_model = create_model(learning_rate)

# Train the model on the normalized training set.
epochs, hist = train_model(my_model, X_train, y_train, epochs, batch_size, validation_split)




Epoch 1/120
15/15 [==============================] - 2s 70ms/step - loss: 1.1280 - accuracy: 0.3311 - val_loss: 1.1430 - val_accuracy: 0.2456
Epoch 2/120
15/15 [==============================] - 0s 25ms/step - loss: 1.0975 - accuracy: 0.3925 - val_loss: 1.0562 - val_accuracy: 0.5000
Epoch 3/120
15/15 [==============================] - 0s 25ms/step - loss: 1.0625 - accuracy: 0.3969 - val_loss: 1.0238 - val_accuracy: 0.4649
Epoch 4/120
15/15 [==============================] - 0s 25ms/step - loss: 1.0438 - accuracy: 0.4474 - val_loss: 1.0047 - val_accuracy: 0.5263
Epoch 5/120
15/15 [==============================] - 0s 23ms/step - loss: 1.0182 - accuracy: 0.4693 - val_loss: 0.9988 - val_accuracy: 0.5351
Epoch 6/120
15/15 [==============================] - 0s 26ms/step - loss: 0.9829 - accuracy: 0.5066 - val_loss: 0.9593 - val_accuracy: 0.6491
Epoch 7/120
15/15 [==============================] - 0s 25ms/step - loss: 0.9718 - accuracy: 0.5461 - val_loss: 0.9631 - val_accuracy: 0.5789
Epoch 

15/15 [==============================] - 0s 23ms/step - loss: 0.4066 - accuracy: 0.8728 - val_loss: 0.6973 - val_accuracy: 0.7456
Epoch 116/120
15/15 [==============================] - 0s 22ms/step - loss: 0.4184 - accuracy: 0.8640 - val_loss: 0.6949 - val_accuracy: 0.7193
Epoch 117/120
15/15 [==============================] - 0s 22ms/step - loss: 0.4148 - accuracy: 0.8596 - val_loss: 0.7137 - val_accuracy: 0.7105
Epoch 118/120
15/15 [==============================] - 0s 23ms/step - loss: 0.3975 - accuracy: 0.8882 - val_loss: 0.7296 - val_accuracy: 0.7018
Epoch 119/120
15/15 [==============================] - 0s 22ms/step - loss: 0.3979 - accuracy: 0.8816 - val_loss: 0.7597 - val_accuracy: 0.6842
Epoch 120/120
15/15 [==============================] - 0s 21ms/step - loss: 0.4014 - accuracy: 0.8772 - val_loss: 0.7629 - val_accuracy: 0.6930
INFO:tensorflow:Assets written to: CNN.model\assets


In [6]:
# # Plot a graph of the metric vs. epochs.
# list_of_metrics_to_plot = ['accuracy', 'val_accuracy']
# plot_curve(epochs, hist, list_of_metrics_to_plot)

In [7]:
# # Plot a graph of the metric vs. epochs.
# list_of_metrics_to_plot = ['loss', 'val_loss']
# plot_curve(epochs, hist, list_of_metrics_to_plot)

In [9]:
import cv2
import tensorflow as tf

CATEGORIES = ["Eating", "Drinking", "Pregnant"]


def prepare(filepath):
    IMG_SIZE = 200  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("CNN.model")

prediction = model.predict([prepare('Testing/pregnant20.jpg')])
print(prediction)  
MaxPosition=np.argmax(prediction)  
prediction_label= CATEGORIES[MaxPosition]
print(prediction_label)

[[1.0659901e-22 0.0000000e+00 1.0000000e+00]]
Pregnant


In [10]:
import cv2
import tensorflow as tf

CATEGORIES = ["Eating", "Drinking", "Pregnant"]


def prepare(filepath):
    IMG_SIZE = 200  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("CNN.model")

prediction = model.predict([prepare('Testing/eating83.jpg')])
MaxPosition=np.argmax(prediction)  
prediction_label= CATEGORIES[MaxPosition]
print(prediction_label)
print(prediction)

Eating
[[1. 0. 0.]]


In [11]:
import cv2
import tensorflow as tf

CATEGORIES = ["Eating", "Drinking", "Pregnant"]


def prepare(filepath):
    IMG_SIZE = 200  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("CNN.model")

prediction = model.predict([prepare('Testing/pregnant48.jpg')])
print(prediction)
MaxPosition=np.argmax(prediction)  
prediction_label= CATEGORIES[MaxPosition]
print(prediction_label) 


[[0. 0. 1.]]
Pregnant
